# What's the Weather Like?

 1. Showcase Temperature (F) vs. Latitude (deg)
 2. Showcase Humidity (% H20) vs. Latitude (deg)
 3. Showcase Cloudiness (% coverage) vs. Latitude (deg)
 4. Showcase Wind Speed (mph) vs. Latitude (deg)
 
These plots are based on 500 unique cities, randomly selected.  Compete city API log in [city.log](./city_log.txt).  Full data in [weather.csv](./weather.csv).

Plotted using Matplotlib and Seaborn.  Uses [citypy](https://github.com/wingchen/citipy) for ease of selecting cities.

In [21]:
from citipy import citipy
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import requests
import seaborn as srn

import json
import logging
import time

srn.set()

In [17]:
logger = logging.getLogger('API calls')
logger.setLevel(logging.INFO)
fh = logging.FileHandler('city.log')
fh.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
logger.addHandler(fh)
logger.addHandler(ch)

In [4]:
lats = np.random.randint(-90, 90, size=500)
longs = np.random.randint(-180, 180, size=500)
coords = pd.DataFrame({"latitude": lats, "longitude": longs})
coords.head()

,latitude,longitude
0,76,169
1,12,86
2,31,175
3,-6,-129
4,8,-70


In [18]:
from secrets import API_KEY
def get_weather_data(coords, time_between=1):
    """Hits OpenWeatherMap API to get weather data for coordinates.
    Coords is a dataframe of 2 columns: Latitude and Longitude.
    Returns a list of json responses.
    """
    results = []
    for ind, row in coords.iterrows():
        query = f"http://api.openweathermap.org/data/2.5/weather?lat={row['latitude']}&lon={row['longitude']}&APPID={API_KEY}"
        city = citipy.nearest_city(row['latitude'], row['longitude'])
        logger.info(f"Call {ind}: {city.city_name} ({query})")
        result = requests.get(query)
        results.append(result.json())
        time.sleep(time_between)
    return results


In [13]:
test = pd.DataFrame({"latitude": [37], "longitude": [-122]})
test_results = get_weather_data(test)
test_results

[{'base': 'stations',
  'clouds': {'all': 1},
  'cod': 200,
  'coord': {'lat': 37, 'lon': -122},
  'dt': 1522184280,
  'id': 5381421,
  'main': {'humidity': 45,
   'pressure': 1023,
   'temp': 293.87,
   'temp_max': 295.15,
   'temp_min': 291.15},
  'name': 'Pasatiempo',
  'sys': {'country': 'US',
   'id': 512,
   'message': 0.164,
   'sunrise': 1522159177,
   'sunset': 1522204017,
   'type': 1},
  'visibility': 16093,
  'weather': [{'description': 'clear sky',
    'icon': '01d',
    'id': 800,
    'main': 'Clear'}],
  'wind': {'deg': 200, 'speed': 3.6}}]

In [22]:
# Commenting out so I don't accidentally hit the API 500 more times.
# full_results = get_weather_data(coords)
# full_results[:3]

[{'base': 'stations',
  'clouds': {'all': 36},
  'cod': 200,
  'coord': {'lat': 76, 'lon': 169},
  'dt': 1522188523,
  'id': 0,
  'main': {'grnd_level': 1025.9,
   'humidity': 88,
   'pressure': 1025.9,
   'sea_level': 1025.86,
   'temp': 251.87,
   'temp_max': 251.87,
   'temp_min': 251.87},
  'name': '',
  'sys': {'message': 0.0049, 'sunrise': 1522086642, 'sunset': 1522137091},
  'weather': [{'description': 'scattered clouds',
    'icon': '03n',
    'id': 802,
    'main': 'Clouds'}],
  'wind': {'deg': 299.501, 'speed': 1.32}},
 {'base': 'stations',
  'clouds': {'all': 0},
  'cod': 200,
  'coord': {'lat': 12, 'lon': 86},
  'dt': 1522188531,
  'id': 0,
  'main': {'grnd_level': 1023.87,
   'humidity': 100,
   'pressure': 1023.87,
   'sea_level': 1023.88,
   'temp': 301.095,
   'temp_max': 301.095,
   'temp_min': 301.095},
  'name': '',
  'sys': {'message': 0.1637, 'sunrise': 1522109722, 'sunset': 1522153624},
  'weather': [{'description': 'clear sky',
    'icon': '01n',
    'id': 800,
 

In [23]:
# Save the data out in case the world blows up
# Commenting out so I don't accidentally overwrite my data
# with open("weather.json", "w") as outfile:
#     json.dump(full_results, outfile)
